In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima.arima import auto_arima
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

def time_series_models_comparison():
    # Load the dataset
    dataset = pd.read_csv("data.csv", parse_dates=["ds"])
    
    # Complete missing dates and fill missing values
    dates = pd.date_range(start="1980-01-01", end="2012-12-01", freq="M")
    dataset = dataset.set_index("ds").reindex(dates).reset_index()
    dataset = dataset.fillna(method="ffill")
    
    # Split the dataset into training and test sets
    trainset = dataset[:-24].copy()
    testset = dataset[-24:].copy()
    
    # Fit ARIMA model
    model_arima = auto_arima(trainset["y"], suppress_warnings=True)
    forecast_arima = model_arima.predict(n_periods=24)
    
    # Fit prophet model
    model_prophet = Prophet(algorithm="Newton")
    model_prophet.fit(trainset[["ds", "y"]].rename(columns={"ds": "ds", "y": "y"}))
    future = model_prophet.make_future_dataframe(periods=24, freq="M")
    forecast_prophet = model_prophet.predict(future)["yhat"].tail(24).values
    
    # Fit exponential smoothing model
    model_ex_smooth = ExponentialSmoothing(trainset["y"])
    model_ex_smooth = model_ex_smooth.fit()
    forecast_ex_smooth = model_ex_smooth.forecast(24)
    
    # Calculate mean squared errors
    mse_arima = mean_squared_error(testset["y"], forecast_arima)
    mse_prophet = mean_squared_error(testset["y"], forecast_prophet)
    mse_ex_smooth = mean_squared_error(testset["y"], forecast_ex_smooth)
    
    # Create a dictionary with the results
    forecasts = pd.DataFrame({"arima": forecast_arima, "prophet": forecast_prophet, "ex_smooth": forecast_ex_smooth}, index=testset.index)
    errors = pd.DataFrame({"model": ["arima", "prophet", "ex_smooth"], "mse": [mse_arima, mse_prophet, mse_ex_smooth]})
    
    result = {"trainset": trainset, "testset": testset, "forecasts": forecasts, "errors": errors}
    
    return result
